# LangGraph Collaboration Patterns
## Content Strategy Decision Engine

Previous notebooks showed **hierarchical** patterns where a supervisor delegates work. But what happens when agents need to truly **collaborate** — analyzing the same problem in parallel, debating opposing viewpoints, or voting on decisions?

This notebook covers three powerful collaboration patterns:

| Pattern | How It Works | Best For |
|---------|-------------|----------|
| **Map-Reduce** | Fan out to N agents in parallel, then reduce results | Parallel analysis, multi-perspective research |
| **Debate** | Agents argue opposing positions in rounds | Decision-making, risk analysis, stress-testing ideas |
| **Voting** | Multiple agents cast structured votes | Group decisions, consensus building, prioritization |

### The Key API: `Send()`

All three patterns use LangGraph's `Send()` API to **fan out** work to multiple agents simultaneously. Instead of routing to one node, `Send()` creates parallel executions — each with its own state payload.

```python
from langgraph.constants import Send

# Instead of routing to one agent:
# return "analyst"

# Fan out to multiple agents simultaneously:
return [
    Send("analyst", {"topic": "market", ...}),
    Send("analyst", {"topic": "tech", ...}),
    Send("analyst", {"topic": "customer", ...}),
]
```

In [ ]:
%pip install -q langgraph langchain langchain-openai langchain-community

In [ ]:
import os
import getpass
import operator
from typing import Annotated, TypedDict, Literal, Any

from pydantic import BaseModel, Field

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.constants import Send

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Setup complete!")

---
# Part A: Map-Reduce Pattern
## Parallel Multi-Perspective Analysis

### The Pattern

Map-Reduce fans out identical analysis tasks to multiple agents, each with a different **perspective**, then combines all results into a single synthesis.

```
                    ┌──────────────┐
                    │  Dispatcher   │
                    └──────┬───────┘
                     Send() │ Fan-out
              ┌────────────┼────────────┐
              ▼            ▼            ▼
        ┌──────────┐ ┌──────────┐ ┌──────────┐
        │  Market   │ │   Tech   │ │ Customer │
        │  Analyst  │ │  Analyst │ │  Analyst │
        └────┬─────┘ └────┬─────┘ └────┬─────┘
              └────────────┼────────────┘
                     Collect │ (operator.add)
                    ┌───────▼──────┐
                    │   Reducer    │
                    └──────────────┘
```

### Key Mechanism: `operator.add` Reducer

When multiple `Send()` calls write to the same list field, the `operator.add` reducer **concatenates** all results. This is how parallel outputs are automatically collected.

```python
class State(TypedDict):
    analyses: Annotated[list, operator.add]  # All analyst outputs merge here
```

In [ ]:
# State for individual analyst work
class AnalystInput(TypedDict):
    topic: str
    perspective: str
    context: str

# Main state that collects all analyses
class MapReduceState(TypedDict):
    topic: str
    analyses: Annotated[list, operator.add]  # Collects outputs from all analysts
    final_synthesis: str

print("Map-Reduce state types defined")
print(f"  MapReduceState fields: {list(MapReduceState.__annotations__.keys())}")
print(f"  Note: 'analyses' uses operator.add reducer for automatic collection")

In [ ]:
PERSPECTIVES = [
    {
        "name": "Market Strategist",
        "perspective": "market_strategy",
        "prompt": """You are a Market Strategist. Analyze this opportunity from a market perspective:
- Market size and growth trajectory
- Competitive dynamics and market gaps
- Go-to-market strategy recommendations
- Market timing considerations

Be specific with data estimates and strategic recommendations. 2-3 paragraphs."""
    },
    {
        "name": "Technology Evaluator", 
        "perspective": "technology",
        "prompt": """You are a Technology Evaluator. Analyze this opportunity from a technology perspective:
- Technical feasibility and maturity of required technologies
- Build vs buy considerations
- Technical risks and mitigation strategies
- Infrastructure and scalability requirements

Be specific with technical assessments. 2-3 paragraphs."""
    },
    {
        "name": "Customer Advocate",
        "perspective": "customer",
        "prompt": """You are a Customer Advocate. Analyze this opportunity from the customer perspective:
- Target customer pain points and willingness to pay
- User experience and adoption barriers
- Customer acquisition and retention strategies
- Customer success metrics and feedback loops

Be specific with customer insights. 2-3 paragraphs."""
    },
]

def dispatcher(state: MapReduceState) -> list:
    """Fan out analysis to multiple perspective agents using Send()."""
    return [
        Send("analyst", {
            "topic": state["topic"],
            "perspective": p["perspective"],
            "context": p["prompt"],
        })
        for p in PERSPECTIVES
    ]

print(f"Dispatcher will fan out to {len(PERSPECTIVES)} analysts: {[p['name'] for p in PERSPECTIVES]}")

In [ ]:
def analyst(state: AnalystInput) -> dict:
    """Individual analyst that processes its assigned perspective."""
    response = llm.invoke([
        SystemMessage(content=state["context"]),
        HumanMessage(content=f"Analyze this opportunity: {state['topic']}"),
    ])
    
    return {
        "analyses": [{
            "perspective": state["perspective"],
            "analysis": response.content,
        }]
    }

def reducer(state: MapReduceState) -> dict:
    """Synthesize all analyst perspectives into a unified recommendation."""
    analyses_text = "\n\n".join([
        f"### {a['perspective'].replace('_', ' ').title()} Analysis:\n{a['analysis']}"
        for a in state["analyses"]
    ])
    
    response = llm.invoke([
        SystemMessage(content="""You are a senior strategy consultant. Synthesize multiple analyst perspectives 
into a unified strategic recommendation. 

Your synthesis should:
1. Identify areas of agreement across perspectives
2. Highlight tensions or trade-offs between perspectives
3. Provide a clear, actionable recommendation
4. Rate the overall opportunity (Strong Opportunity / Moderate Opportunity / Weak Opportunity / Pass)

Be concise but comprehensive. Use specific data points from the analyses."""),
        HumanMessage(content=f"Topic: {state['topic']}\n\nAnalyst Reports:\n{analyses_text}"),
    ])
    
    return {"final_synthesis": response.content}

print("Analyst and reducer nodes defined")

In [ ]:
mr_graph = StateGraph(MapReduceState)

mr_graph.add_node("analyst", analyst)
mr_graph.add_node("reducer", reducer)

# Dispatcher uses Send() — it's a conditional edge that returns Send objects
mr_graph.add_conditional_edges(START, dispatcher, ["analyst"])
mr_graph.add_edge("analyst", "reducer")
mr_graph.add_edge("reducer", END)

mr_app = mr_graph.compile()
print("Map-Reduce graph compiled!")

In [ ]:
from IPython.display import display, Image

try:
    display(Image(mr_app.get_graph().draw_mermaid_png()))
except Exception:
    print(mr_app.get_graph().draw_mermaid())

In [ ]:
mr_result = mr_app.invoke({
    "topic": "AI-powered healthcare diagnostics: Building a platform that uses computer vision and NLP to assist radiologists in detecting early-stage cancers from medical imaging, starting with lung CT scans.",
    "analyses": [],
    "final_synthesis": "",
})

print("=" * 80)
print("MAP-REDUCE: Multi-Perspective Opportunity Analysis")
print("=" * 80)

print(f"\nPerspectives analyzed: {len(mr_result['analyses'])}")
for analysis in mr_result["analyses"]:
    print(f"\n{'─' * 60}")
    print(f"  {analysis['perspective'].replace('_', ' ').upper()}")
    print(f"{'─' * 60}")
    print(f"  {analysis['analysis'][:400]}...")

print(f"\n{'=' * 60}")
print("UNIFIED SYNTHESIS")
print(f"{'=' * 60}")
print(mr_result["final_synthesis"][:800])

---
# Part B: Debate Pattern
## Structured Argumentation

### The Pattern

Two agents argue **opposing positions** over multiple rounds, moderated by a judge who decides when the debate has reached a conclusion.

```
moderator_intro → bull_agent → bear_agent → moderator_eval ─┐
                      ▲                                       │
                      └──── (another round) ──────────────────┘
                                                              │
                                              (debate over) ──┤
                                                              ▼
                                                       final_summary
```

### Why Debate?

- **Stress-tests ideas** — Forces explicit consideration of counterarguments
- **Reduces bias** — Each agent is incentivized to find weaknesses in the other's position
- **Creates better decisions** — The synthesis of bull/bear cases is richer than a single analysis
- **Transparent reasoning** — Decision-makers can see both sides of the argument

### Key Mechanism: Multi-Round Loops

The moderator evaluates after each round and decides whether to continue or conclude. The `round_number` counter prevents infinite debates.

In [ ]:
class DebateState(TypedDict):
    messages: Annotated[list, add_messages]
    topic: str
    round_number: int
    max_rounds: int
    bull_arguments: Annotated[list, operator.add]
    bear_arguments: Annotated[list, operator.add]
    debate_status: str  # "ongoing" or "concluded"
    final_verdict: str

print("DebateState defined with round tracking and argument collection")

In [ ]:
def moderator_intro(state: DebateState) -> dict:
    """Set up the debate with the topic and rules."""
    response = llm.invoke([
        SystemMessage(content="""You are a debate moderator. Introduce the topic and frame the key question.
Set up what each side should address. Keep it to 2-3 sentences."""),
        HumanMessage(content=f"Debate topic: {state['topic']}"),
    ])
    
    return {
        "messages": [AIMessage(content=f"[Moderator] {response.content}")],
        "round_number": 1,
        "debate_status": "ongoing",
    }

def bull_agent(state: DebateState) -> dict:
    """Argues the bullish/positive case."""
    round_num = state.get("round_number", 1)
    bear_args = state.get("bear_arguments", [])
    
    if round_num == 1:
        prompt = f"""You are the BULL (advocate FOR the proposal). This is Round {round_num}.
Make your opening argument FOR this proposal. Be specific, cite potential benefits, and make a compelling case.
Topic: {state['topic']}"""
    else:
        last_bear = bear_args[-1] if bear_args else "No counter-arguments yet."
        prompt = f"""You are the BULL (advocate FOR the proposal). This is Round {round_num}.
The BEAR just argued: {last_bear}

Counter their arguments and strengthen your case. Address their specific points.
Topic: {state['topic']}"""
    
    response = llm.invoke([
        SystemMessage(content=prompt),
        *state["messages"],
    ])
    
    return {
        "messages": [AIMessage(content=f"[Bull - Round {round_num}] {response.content}")],
        "bull_arguments": [response.content],
    }

def bear_agent(state: DebateState) -> dict:
    """Argues the bearish/negative case."""
    round_num = state.get("round_number", 1)
    bull_args = state.get("bull_arguments", [])
    last_bull = bull_args[-1] if bull_args else "No arguments yet."
    
    response = llm.invoke([
        SystemMessage(content=f"""You are the BEAR (advocate AGAINST the proposal). This is Round {round_num}.
The BULL just argued: {last_bull}

Counter their arguments and make the case AGAINST. Identify risks, costs, and alternatives.
Topic: {state['topic']}"""),
        *state["messages"],
    ])
    
    return {
        "messages": [AIMessage(content=f"[Bear - Round {round_num}] {response.content}")],
        "bear_arguments": [response.content],
    }

class DebateEvaluation(BaseModel):
    """Moderator's evaluation of the current debate round."""
    should_continue: bool = Field(description="Whether another round of debate would add value")
    reasoning: str = Field(description="Why the debate should continue or end")
    leading_side: Literal["bull", "bear", "balanced"] = Field(description="Which side currently has the stronger argument")

moderator_eval_llm = llm.with_structured_output(DebateEvaluation)

def moderator_eval(state: DebateState) -> dict:
    """Evaluate the debate round and decide whether to continue."""
    round_num = state.get("round_number", 1)
    max_rounds = state.get("max_rounds", 3)
    
    # Force conclusion if at max rounds
    if round_num >= max_rounds:
        return {
            "messages": [AIMessage(content=f"[Moderator] Maximum rounds ({max_rounds}) reached. Moving to final verdict.")],
            "debate_status": "concluded",
        }
    
    evaluation = moderator_eval_llm.invoke([
        SystemMessage(content="""You are a debate moderator. Evaluate the current round:
- Are both sides making substantive, non-repetitive arguments?
- Have the key issues been thoroughly explored?
- Would another round add meaningful new perspectives?

If arguments are becoming repetitive or all key points have been made, the debate should conclude."""),
        *state["messages"],
    ])
    
    status = "ongoing" if evaluation.should_continue else "concluded"
    
    return {
        "messages": [AIMessage(content=f"[Moderator] Round {round_num} evaluation: {evaluation.reasoning} (Leading: {evaluation.leading_side})")],
        "debate_status": status,
        "round_number": round_num + 1,
    }

def final_summary(state: DebateState) -> dict:
    """Synthesize the debate into a final verdict."""
    bull_args = state.get("bull_arguments", [])
    bear_args = state.get("bear_arguments", [])
    
    response = llm.invoke([
        SystemMessage(content="""You are the debate moderator delivering the final verdict.

Synthesize both sides' arguments into a balanced recommendation:
1. Strongest bull arguments
2. Strongest bear arguments  
3. Key trade-offs
4. YOUR VERDICT: Recommend FOR, AGAINST, or CONDITIONAL (with specific conditions)

Be decisive — don't just summarize, take a position."""),
        HumanMessage(content=f"""Topic: {state['topic']}

BULL Arguments (all rounds):
{chr(10).join(f'Round {i+1}: {arg[:300]}' for i, arg in enumerate(bull_args))}

BEAR Arguments (all rounds):
{chr(10).join(f'Round {i+1}: {arg[:300]}' for i, arg in enumerate(bear_args))}"""),
    ])
    
    return {
        "messages": [AIMessage(content=f"[Final Verdict] {response.content}")],
        "final_verdict": response.content,
    }

print("Debate nodes defined: moderator_intro, bull_agent, bear_agent, moderator_eval, final_summary")

In [ ]:
def route_debate(state: DebateState) -> str:
    """Route based on debate status."""
    if state.get("debate_status") == "concluded":
        return "final_summary"
    return "bull_agent"

debate_graph = StateGraph(DebateState)

debate_graph.add_node("moderator_intro", moderator_intro)
debate_graph.add_node("bull_agent", bull_agent)
debate_graph.add_node("bear_agent", bear_agent)
debate_graph.add_node("moderator_eval", moderator_eval)
debate_graph.add_node("final_summary", final_summary)

debate_graph.add_edge(START, "moderator_intro")
debate_graph.add_edge("moderator_intro", "bull_agent")
debate_graph.add_edge("bull_agent", "bear_agent")
debate_graph.add_edge("bear_agent", "moderator_eval")
debate_graph.add_conditional_edges("moderator_eval", route_debate, {
    "bull_agent": "bull_agent",
    "final_summary": "final_summary",
})
debate_graph.add_edge("final_summary", END)

debate_app = debate_graph.compile()
print("Debate graph compiled!")

In [ ]:
try:
    display(Image(debate_app.get_graph().draw_mermaid_png()))
except Exception:
    print(debate_app.get_graph().draw_mermaid())

In [ ]:
debate_result = debate_app.invoke({
    "messages": [],
    "topic": "Should a Fortune 500 company invest $100M in building its own proprietary foundation model, rather than using existing models from OpenAI, Anthropic, or open-source alternatives?",
    "round_number": 0,
    "max_rounds": 3,
    "bull_arguments": [],
    "bear_arguments": [],
    "debate_status": "",
    "final_verdict": "",
})

print("=" * 80)
print("DEBATE: Build vs Buy Foundation Models ($100M)")
print("=" * 80)

for msg in debate_result["messages"]:
    if isinstance(msg, AIMessage):
        print(f"\n{msg.content[:400]}")
        print()

print(f"\nTotal rounds: {debate_result.get('round_number', 0) - 1}")
print(f"Bull arguments: {len(debate_result.get('bull_arguments', []))}")
print(f"Bear arguments: {len(debate_result.get('bear_arguments', []))}")

---
# Part C: Voting Pattern
## Multi-Stakeholder Decision Making

### The Pattern

Multiple agents with different **roles and priorities** independently evaluate a proposal and cast structured votes. A tally agent counts votes and announces the decision.

```
                    ┌───────────────────┐
                    │ Proposal Presenter │
                    └────────┬──────────┘
                       Send() │ Fan-out
              ┌──────────────┼──────────────┐
              ▼              ▼              ▼
        ┌──────────┐  ┌──────────┐  ┌──────────┐
        │  Voter   │  │  Voter   │  │  Voter   │
        │  (CFO)   │  │  (CTO)   │  │  (CPO)   │
        └────┬─────┘  └────┬─────┘  └────┬─────┘
              └──────────────┼──────────────┘
                       Collect │ (operator.add)
                    ┌────────▼─────────┐
                    │      Tally       │
                    └────────┬─────────┘
                    ┌────────▼─────────┐
                    │    Announce      │
                    └──────────────────┘
```

### Key Mechanism: Structured Output Votes

Each voter returns a Pydantic model with their vote and reasoning. This ensures consistent, parseable results.

In [ ]:
class Vote(BaseModel):
    """A structured vote from a stakeholder."""
    voter_role: str = Field(description="The role of the voter")
    vote: Literal["APPROVE", "REJECT", "CONDITIONAL"] = Field(description="The vote")
    confidence: float = Field(description="Confidence in this vote, 0.0 to 1.0", ge=0.0, le=1.0)
    reasoning: str = Field(description="Detailed reasoning for the vote")
    conditions: str = Field(default="", description="Conditions if vote is CONDITIONAL")

class VoterInput(TypedDict):
    proposal: str
    voter_role: str
    voter_prompt: str

class VotingState(TypedDict):
    proposal: str
    votes: Annotated[list, operator.add]  # Collects Vote objects from all voters
    tally_result: str
    final_decision: str

vote_llm = llm.with_structured_output(Vote)

print("Voting state and Vote model defined")

In [ ]:
VOTERS = [
    {
        "role": "CFO (Chief Financial Officer)",
        "prompt": """You are the CFO. Evaluate this proposal from a FINANCIAL perspective:
- ROI and payback period
- Budget impact and opportunity cost
- Revenue impact and financial risk
- Cash flow implications

Vote APPROVE, REJECT, or CONDITIONAL. Be specific about financial thresholds."""
    },
    {
        "role": "CTO (Chief Technology Officer)",
        "prompt": """You are the CTO. Evaluate this proposal from a TECHNOLOGY perspective:
- Technical feasibility and complexity
- Engineering resource requirements
- Technical debt and maintenance burden
- Innovation potential and competitive advantage

Vote APPROVE, REJECT, or CONDITIONAL. Be specific about technical requirements."""
    },
    {
        "role": "CPO (Chief Product Officer)",
        "prompt": """You are the CPO. Evaluate this proposal from a PRODUCT perspective:
- Customer demand and market fit
- User experience impact
- Product roadmap alignment
- Competitive differentiation

Vote APPROVE, REJECT, or CONDITIONAL. Be specific about product metrics."""
    },
]

def proposal_presenter(state: VotingState) -> list:
    """Present the proposal to all voters simultaneously using Send()."""
    return [
        Send("voter", {
            "proposal": state["proposal"],
            "voter_role": v["role"],
            "voter_prompt": v["prompt"],
        })
        for v in VOTERS
    ]

def voter(state: VoterInput) -> dict:
    """Individual voter casts a structured vote."""
    vote = vote_llm.invoke([
        SystemMessage(content=f"""{state['voter_prompt']}

IMPORTANT: Your voter_role field MUST be exactly: {state['voter_role']}"""),
        HumanMessage(content=f"Proposal to evaluate:\n\n{state['proposal']}"),
    ])
    
    # Ensure role is set correctly
    vote.voter_role = state["voter_role"]
    
    return {
        "votes": [vote.model_dump()],
    }

def tally(state: VotingState) -> dict:
    """Count votes and determine the outcome."""
    votes = state.get("votes", [])
    
    approve_count = sum(1 for v in votes if v["vote"] == "APPROVE")
    reject_count = sum(1 for v in votes if v["vote"] == "REJECT")
    conditional_count = sum(1 for v in votes if v["vote"] == "CONDITIONAL")
    
    total = len(votes)
    avg_confidence = sum(v["confidence"] for v in votes) / total if total > 0 else 0
    
    tally_text = f"""VOTE TALLY:
  APPROVE: {approve_count}/{total}
  REJECT: {reject_count}/{total}  
  CONDITIONAL: {conditional_count}/{total}
  Average Confidence: {avg_confidence:.0%}

INDIVIDUAL VOTES:"""
    
    for v in votes:
        tally_text += f"\n  {v['voter_role']}: {v['vote']} (confidence: {v['confidence']:.0%})"
        tally_text += f"\n    Reasoning: {v['reasoning'][:200]}"
        if v.get("conditions"):
            tally_text += f"\n    Conditions: {v['conditions'][:150]}"
    
    return {"tally_result": tally_text}

def announce(state: VotingState) -> dict:
    """Announce the final decision based on votes."""
    response = llm.invoke([
        SystemMessage(content="""You are the board secretary announcing the voting results.
Based on the vote tally, announce the final decision:
- Majority APPROVE → "APPROVED"
- Majority REJECT → "REJECTED"  
- Mixed/conditional → "APPROVED WITH CONDITIONS" (list the conditions)

Be formal and specific. Include the vote count and key conditions."""),
        HumanMessage(content=f"Proposal: {state['proposal']}\n\n{state['tally_result']}"),
    ])
    
    return {"final_decision": response.content}

print(f"Voting nodes defined with {len(VOTERS)} voters: {[v['role'] for v in VOTERS]}")

In [ ]:
voting_graph = StateGraph(VotingState)

voting_graph.add_node("voter", voter)
voting_graph.add_node("tally", tally)
voting_graph.add_node("announce", announce)

voting_graph.add_conditional_edges(START, proposal_presenter, ["voter"])
voting_graph.add_edge("voter", "tally")
voting_graph.add_edge("tally", "announce")
voting_graph.add_edge("announce", END)

voting_app = voting_graph.compile()
print("Voting graph compiled!")

In [ ]:
try:
    display(Image(voting_app.get_graph().draw_mermaid_png()))
except Exception:
    print(voting_app.get_graph().draw_mermaid())

In [ ]:
voting_result = voting_app.invoke({
    "proposal": """PROPOSAL: Pivot from B2B SaaS to B2C Consumer App

Our enterprise SaaS product has $5M ARR but growth has slowed to 15% YoY. We propose pivoting 
to a consumer-facing app targeting individual users.

Key details:
- Estimated pivot cost: $3M over 12 months
- Current B2B team: 30 engineers, 10 sales
- Consumer market TAM: 10x larger than our B2B TAM
- Would require hiring 15 new consumer-focused engineers
- B2B revenue would decline ~40% during transition
- Consumer product would be freemium with $9.99/month premium tier
- Need 500K MAU to break even on consumer side
- Competitors: 3 well-funded startups already in consumer space""",
    "votes": [],
    "tally_result": "",
    "final_decision": "",
})

print("=" * 80)
print("VOTING: B2B to B2C Pivot Decision")
print("=" * 80)

print(f"\n{voting_result['tally_result']}")

print(f"\n{'=' * 60}")
print("FINAL DECISION")
print(f"{'=' * 60}")
print(voting_result["final_decision"])

## Key Takeaways

### Pattern Comparison

| Aspect | Map-Reduce | Debate | Voting |
|--------|-----------|--------|--------|
| Agent interaction | Independent (parallel) | Sequential (adversarial) | Independent (parallel) |
| `Send()` usage | Fan-out to analysts | Not needed (sequential) | Fan-out to voters |
| Output collection | `operator.add` list | Separate bull/bear lists | `operator.add` list |
| Best for | Multi-perspective analysis | Stress-testing decisions | Group consensus |
| Rounds | Single pass | Multiple rounds | Single pass |
| Structured output | Optional | Optional | Essential (Vote model) |

### Key API Summary

```python
# Send() — fan-out to parallel agents
from langgraph.constants import Send

def dispatcher(state):
    return [Send("node_name", {"key": "value"}) for item in items]

# operator.add — collect parallel results
from operator import add
class State(TypedDict):
    results: Annotated[list, operator.add]  # Auto-concatenates

# Structured output — typed agent responses
class Vote(BaseModel):
    vote: Literal["YES", "NO"]
    reasoning: str

vote_llm = llm.with_structured_output(Vote)
```

### When to Use Each Pattern

- **Map-Reduce**: You need the same type of analysis from different perspectives/data sources
- **Debate**: You need to stress-test a decision, especially high-stakes ones with unclear trade-offs
- **Voting**: You need multiple stakeholders to weigh in with their domain expertise, and a clear decision mechanism

### Combining Patterns

These patterns compose well:
- **Debate + Voting**: Have agents debate, then have a separate panel vote on the winner
- **Map-Reduce + Voting**: Gather parallel analyses, then have each analyst vote on the recommendation
- **Hierarchical + Map-Reduce**: Each sub-team uses map-reduce internally

### Next Steps
- **Custom State Machines** → When you need explicit lifecycle stages with complex branching and retry logic
- **Human in the Loop** → When you need human approval at certain decision points